# 디폴드 값 및 위치, 키워드 인자의 차이점을 설명하시오.
- 파이썬 함수는 다양한 방식으로 인자를 전달할 수 있음.

In [2]:
def get_analog(channel, sample=10, calibration=0.2, rms= True):
    print(channel, sample, calibration, rms)
 
get_analog (1)
get_analog (1, 100)
get_analog (1, rms=False)
get_analog (calibration=0.5, channel=1)


1 10 0.2 True
1 100 0.2 True
1 10 0.2 False
1 10 0.5 True


In [ ]:
# print()처럼 인자를 가변으로 전달하는 함수를 구현하시오.
- 함수의 매개변수와 인자는 여러 개의 인자를 묶어 반복 가능한 객체로 전달하는 패킹과 이를 다시 풀어놓은 언패킹 지원



In [4]:
def get_analog(channel, *args, **kwargs):
    print(channel, len(args), args, len(kwargs), kwargs)
    
get_analog (1)
get_analog (1, 2, 3)
get_analog (1, *[2, 3])
get_analog(1, sample=1, calibration=2, rms= True)
get_analog(1, **{'sample':1, 'calibration':2, 'rms': True})
get_analog(1, 2, 3, sample=1, calibration=2, rms= True)

1 0 () 0 {}
1 2 (2, 3) 0 {}
1 2 (2, 3) 0 {}
1 0 () 3 {'sample': 1, 'calibration': 2, 'rms': True}
1 0 () 3 {'sample': 1, 'calibration': 2, 'rms': True}
1 2 (2, 3) 3 {'sample': 1, 'calibration': 2, 'rms': True}


# 조건을 만족하지 않으면 예외를 던지는 함수를 구현하시오.
- raise는 예외 객체를 호출한 쪽으로 전달함

In [10]:
from random import shuffle, randint

def get_analog(channel, *args, **kwargs):
    if channel < 0 or channel > 15:
        raise ValueError ("Value out of range")
    if len(args) > 3:
        raise SyntaxError ("Invalied argument")

    pseudo_data = [i for i in range(1, 45 + 1)]
    shuffle(pseudo_data)

    return pseudo_data[randint (1, 45)]


In [11]:
try:
    ret1 = get_analog(1)
    ret2 = get_analog(1, 1, 2, 3)
except ValueError as e:
    print(e)
except SyntaxError as e:
    print(e)
else:
    print(ret1, ret2)

33 29


# 클래스처럼 상태를 유지하는 함수를 구현하시오.
- 클로저(closure)는 일급함수(first-class function)의 이름 바인딩 기술로 내부함수 프리 자유변수(free variable)와 함께 저장


In [12]:
def acculumator(do_abs=False):
    sum=0

    def wrapper(n):
        nonlocal sum
        sum+=abs(n) if do_abs else n
        return sum
        
    return wrapper

adder = acculumator(True)
adder(10)
adder(-20)
x=adder(30)
print(x)

60


# 함수를 인자로 실행 시간과 함께 호출하는 함수를 구현하시오.
- 데코레이터(decorator)는 함수를 인자로 받아 실행하는 클로저의 이름 바인딩을 엣(@) 기호로 표현

In [13]:
from time import time

def run_time(func):
    def wrapper(*args, **kwargs):
        start=time()
        t=func(*args, **kwargs)
        print("run time:", time()-start)
        return t
    return wrapper

@run_time
def fibo(n):
    curr, next=0, 1
    for _ in range(n):
        curr, next=next, curr+next
    return curr

print(fibo(10))

run time: 2.86102294921875e-06
55


# Class로 센서 id와 값을 저장하는 구조체를 정의하시오.
- 클래스는 C언어의 구조체 확장으로 멤버 변수를 인스턴스 속성(attribute)이라 함

'''
typedef struct _SensorData {
    int id;
    int value;
} SensorData;

SensorData d1;
SensorData d2;

d1.id = 0x01;
d1.value = 10;
d2.id = 0x02;
d2.value = 20;

printf ("%d: %d", d1.id, d1.value);
printf ("%d: %d", d2.id, d1=2.value);
'''
c 언어 구조체 참고 코드

In [14]:
class SensorData:
    def __init__(self, id=None, value=None):
        self.id = id
        self.value = value

d1 = SensorData()
d2 = SensorData(0x02, 20)

d1.id = 0x01; d1.value = 10

print("%d: %d"%(d1.id, d1.value))
print("%d: %d"%(d2.id, d2.value))

1: 10
2: 20


# __new__()와 __init__()의 차이점을 설명하시오.
- 클래스는 인스턴스 속성 외에 메소드(method)(멤버함수)를 함께 정의할 수 있음

In [30]:
class SensorData:
    def __new__(cls):
        pass
        
    def __init__(self, id=None, value=None):
        self.id = id
        self.value = value

data1 = SensorData()
print(type(data1))

data2 = object.__new__(SensorData)
data2.__init__(0x01, 10)

<class 'NoneType'>


In [34]:
class SensorData:
    def __new__(cls, *args, **kwargs):
        inst = object.__new__(cls)
        return inst
        
    def __init__(self, id, value):
        self.id = id
        self.value = value

data1 = SensorData(1, 2)
print(type(data1))

data2 = SensorData(0x01, 10)

<class '__main__.SensorData'>


# 인스턴스 수를 최대 2개로 제한하는 클래스를 구현하시오.
- __new__() 메소드의 첫 번째 매개변수인 cls는 모든 인스턴스가 공유하는 클래스 객체를 가리킴

In [43]:
class SensorData:
    __instance = []
    __limit = 2

    def __new__(cls, *args, **kwargs):
        if len(cls.__instance) >= cls.__limit:
            raise RuntimeError("Could not create instance")

        inst = object.__new__(cls)
        cls.__instance.append(inst)

        return inst

    def __init__(self, id=None, value=None):
        self.id = id
        self.value = value

    def __del__(self):
        SensorData.__instance.remove(self)

In [44]:
try:
    data1 = SensorData()
    data2 = SensorData(0x01, 10)
    #data3 = SensorData(0x02, 20) #예외발생
except RuntimeError as e:
    print(e.args[0])
else:
    print(data1.id, data1.value)
    print(data2.id, data2.value)

None None
1 10
